In [0]:
!ls
# download dataset
!gdown https://drive.google.com/uc?id=1bqT04uGQKVfbBd9Ru_Wtku8BqhCKY1Vo

mnist_data.mat	sample_data
Downloading...
From: https://drive.google.com/uc?id=1bqT04uGQKVfbBd9Ru_Wtku8BqhCKY1Vo
To: /content/mnist_data.mat
88.7MB [00:00, 243MB/s] 


In [0]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

In [0]:
#get the dataset
data = scipy.io.loadmat("mnist_data.mat") 
#type(data)

In [0]:
#convert dataset to numpy arrays
tr_x = np.array(data["trX"])
tr_y = np.array(data["trY"])
ts_x = np.array(data["tsX"])
ts_y = np.array(data["tsY"])

In [0]:
#func for visualizing any 1 image
def visualizing_one_img(pixels): #input any ts_x or tr_x imag into this function
  #pixels = ts_x[1500]
  #pixels = np.array(pixels) #if not already converted to np array
  #print(len(pixels)) # 784 pixels per image

  # reshaping to 28x28
  pixels = pixels.reshape((28, 28))

  # plotting
  plt.imshow(pixels, cmap='gray')
  plt.show()
  return None

In [0]:
#TRAINING data
#this variable is where the input features (mean and std-dev) of every image will be stored
tr_x_avg_stddev = []

for index, img in enumerate(tr_x): #enumerate to get both index and array
  average_val = np.mean(img)
  stddev_val = np.std(img)
  tr_x_avg_stddev.append([average_val, stddev_val])

tr_x_avg_stddev = np.array(tr_x_avg_stddev)
print

<function print>

In [0]:
#doing the same for TEST data
ts_x_avg_stddev = []
for index, img in enumerate(ts_x): #enumerate to get both index and array
  average_val = np.mean(img)
  stddev_val = np.std(img)
  ts_x_avg_stddev.append([average_val, stddev_val])

ts_x_avg_stddev = np.array(ts_x_avg_stddev)

In [0]:
##########################################################################################################################
#--------------------------------------------------LOGISTIC REGRESSION----------------------------------------------------
##########################################################################################################################

In [0]:
#function for implementing the logistic function
def logistic_func(wghts, input_x):
  return 1.0/(1 + np.exp(-np.dot(input_x, wghts.T))) 

#function for calculating the gradient of the log likelyhood function directly using formula
def log_gradient(wghts, input_x, y):
  return np.dot((logistic_func(wghts, input_x) - y.reshape(input_x.shape[0], -1)).T, input_x)

#fucntion for updating weights
def logistic_regression(input_x, y, wghts, lr=.001):
  #running the gradient ascent for 1000 iterations
  for i in range(0,1000):
    wghts = wghts - (lr * log_gradient(wghts, input_x, y)) 

  return wghts

#fucntion for applying the threshold and giving final output
def predict_logistic_regression_values(wghts, input_x):
  output_val = logistic_func(wghts, input_x)
  for i, val in enumerate(output_val):
    if val >= 0.5:
      output_val[i] = 1
    else:
      output_val[i] = 0
  return output_val


In [0]:
init_wghts = np.matrix(np.random.rand(tr_x_avg_stddev.shape[1]))
wghts = np.matrix(np.zeros(tr_x_avg_stddev.shape[1]))

wghts = logistic_regression(tr_x_avg_stddev, tr_y, init_wghts)
#print(wghts)

In [0]:
#function to calculate the confusion matrix from given true values and predicted values
def compute_confusion_matrix(true, pred):
  #the vlaues in the matrix
  #row 0 [ correct pred 7      wrong pred 7]
  #row 1 [ wring pred 8        correct pred 8]
  result = np.array([[0,0],[0,0]])
  for i, val in enumerate(true):
    if val == 0:
      if pred[i] == 0: 
        result[0][0] += 1
      else:
        result[0][1] += 1
    elif val == 1:
      if pred[i] == 0:
        result[1][0] += 1
      else:
        result[1][1] += 1
  return result

In [0]:
#calculating accuracy from confusion matrix
#on test data
y_pred = predict_logistic_regression_values(wghts, ts_x_avg_stddev)
#cm = metrics.confusion_matrix(ts_y.T, y_pred)
cm = compute_confusion_matrix(ts_y.T, y_pred)
#print(cm)
accuracy = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
accuracy_class0 = (cm[0][0]) / (cm[0][0] + cm[0][1])
accuracy_class1 = (cm[1][1]) / (cm[1][0] + cm[1][1])
print("LOGISTIC REGRESSION")
print("accuracy for 7:", accuracy_class0)
print("accuracy for 8:", accuracy_class1)
print("accuracy:", accuracy)
'''
#on training data
y_pred_tr = predict_logistic_regression_values(wghts, tr_x_avg_stddev)
#cm_tr = metrics.confusion_matrix(tr_y.T, y_pred_tr)
cm_tr = compute_confusion_matrix(tr_y.T, y_pred_tr)
print(cm_tr)
accuracy_tr = (cm_tr[0][0] + cm_tr[1][1])/(cm_tr[0][0] + cm_tr[0][1] + cm_tr[1][0] + cm_tr[1][1])
print(accuracy_tr)
'''

LOGISTIC REGRESSION
accuracy for 7: 0.8005836575875487
accuracy for 8: 0.6735112936344969
accuracy: 0.7387612387612388


'\n#on training data\ny_pred_tr = predict_logistic_regression_values(wghts, tr_x_avg_stddev)\n#cm_tr = metrics.confusion_matrix(tr_y.T, y_pred_tr)\ncm_tr = compute_confusion_matrix(tr_y.T, y_pred_tr)\nprint(cm_tr)\naccuracy_tr = (cm_tr[0][0] + cm_tr[1][1])/(cm_tr[0][0] + cm_tr[0][1] + cm_tr[1][0] + cm_tr[1][1])\nprint(accuracy_tr)\n'

In [0]:
##########################################################################################################################
#------------------------------------------------------NAIVE BAYES--------------------------------------------------------
##########################################################################################################################

In [0]:
zero_count = 0
one_count = 0
for i in tr_y.T:
  if i[0] == 0:
    zero_count = zero_count + 1
  else:
    one_count = one_count + 1

#probablities of both the output lables
zero_prob = zero_count/len(tr_y.T) #prob of 7s
one_prob = one_count/len(tr_y.T) #prob of 8s

In [0]:
#tr_x_avg_stddev --- index 0 is avg; 1 is stddev

#variable nomenclature = class_feature_mean/var

#MEANS
#for zero/7 class
zero_avg_mean = 0
zero_stddev_mean = 0
#for one/8 class
one_avg_mean = 0
one_stddev_mean = 0

#VARIANCES
#for zero/7 class
zero_avg_var = 0
zero_stddev_var = 0
#for one/8 class
one_avg_var = 0
one_stddev_var = 0

#for class zero/7
#tr_x_avg_stddev.T is cut from 0 to zero_count
#MEANS
zero_avg_mean = np.mean(tr_x_avg_stddev.T[0][:zero_count])
zero_stddev_mean = np.mean(tr_x_avg_stddev.T[1][:zero_count])
#VARIANCES
zero_avg_var = np.var(tr_x_avg_stddev.T[0][:zero_count])
zero_stddev_var = np.var(tr_x_avg_stddev.T[1][:zero_count])

#for class one/8
#tr_x_avg_stddev.T is cut from zero_count to end of matrix
#MEANS
one_avg_mean = np.mean(tr_x_avg_stddev.T[0][zero_count:])
one_stddev_mean = np.mean(tr_x_avg_stddev.T[1][zero_count:])
#VARIANCES
one_avg_var = np.var(tr_x_avg_stddev.T[0][zero_count:]) #these values are = 0?
one_stddev_var = np.var(tr_x_avg_stddev.T[1][zero_count:]) #these values are = 0?

In [0]:
#variable nomenclature = class_feature_mean/var

#-------for zero/7 class
print("distribution parameters for class-zero/digit 7")
#distribution parameters for the avg
print("mean for 'average':", zero_avg_mean)
print("variance for 'average':", zero_avg_var)
#distribution parameters for the avg
print("mean for 'std-deviation':", zero_stddev_mean)
print("variance for 'std-deviation':", zero_stddev_var)
#-------
#-------for class one/8
print("distribution parameters for class-one/digit 8")
#distribution parameters for the avg
print("mean for 'average':", one_avg_mean)
print("variance for 'average':", one_avg_var)
#distribution parameters for the avg
print("mean for 'std-deviation':", one_stddev_mean)
print("variance for 'std-deviation':", one_stddev_var)
#-------

distribution parameters for class-zero/digit 7
mean for 'average': 0.11452769775108769
variance for 'average': 0.0009383442174894415
mean for 'std-deviation': 0.28755656517748474
variance for 'std-deviation': 0.0014593227954204646
distribution parameters for class-one/digit 8
mean for 'average': 0.1501559818936975
variance for 'average': 0.0014924691579640671
mean for 'std-deviation': 0.3204758364888714
variance for 'std-deviation': 0.001596807543708563


In [0]:
#function for calulating p(x|y)
def p_of_x_given_y(x, mean_y, variance_y):
    # implementation of the probability density function fro gaussian normal distribution
    p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
    return p

In [0]:

#function for the implementation of the naive bayes formula
def naive_bayes(feature_avg, feature_stddev):
  #probability of input being in class zero/7
  input_is_in_zero = zero_prob * p_of_x_given_y(feature_avg, zero_avg_mean, zero_avg_var) * p_of_x_given_y(feature_stddev, zero_stddev_mean, zero_stddev_var)
  #probability of input being in class one/8
  input_is_in_one = one_prob * p_of_x_given_y(feature_avg, one_avg_mean, one_avg_var) * p_of_x_given_y(feature_stddev, one_stddev_mean, one_stddev_var)

  #classify
  if input_is_in_one > input_is_in_zero:
    return 1
  else:
    return 0

#function for giving out the final naive bayes predicted values
def predict_naive_bayes_values(input_x):
  output_vector = []
  for input_img_features in input_x:
    output_vector.append(naive_bayes(input_img_features[0], input_img_features[1]))
  return np.array(output_vector)

In [0]:
#calculating accuracy from the confusion matrix
#on test data
y_pred = predict_naive_bayes_values(ts_x_avg_stddev)
cm = compute_confusion_matrix(ts_y.T, y_pred)
#print(cm)
accuracy = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[0][1] + cm[1][0] + cm[1][1])
accuracy_class0 = (cm[0][0]) / (cm[0][0] + cm[0][1])
accuracy_class1 = (cm[1][1]) / (cm[1][0] + cm[1][1])
print("NAIVE BAYES")
print("accuracy for 7:", accuracy_class0)
print("accuracy for 8:", accuracy_class1)
print("accuracy:", accuracy)

NAIVE BAYES
accuracy for 7: 0.7597276264591439
accuracy for 8: 0.6273100616016427
accuracy: 0.6953046953046953
